# Language processing & Transformer

음성 AI를 위한 자연어 처리와 Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. 텍스트 전처리 과정 이해
    - tokenizing
    - cleaning
2. Multi-head attention 및 self-attention 구현.
3. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 install & import

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 14.6 MB/s eta 0:00:00


In [2]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm
import re
import torch
import math

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

## Req. 1-1 텍스트 전처리

주어진 문장 5개를 cleaning, tokenizing 한 뒤 정수 인코딩 하시오.  

원하는 다른 tokenizer를 사용해도 좋습니다.

In [3]:
sentences = [["안녕하세요 음성 AI 실!@습에 오신 것을 환영#$^&@$&$합니다."], ["이네들은 7895435너무나 멀리 있습니다."], 
["계절이 지나가는 하늘에는가을로 가&^%@!$!^득 차 있습니다."], ["아직 나의 청!@$!%춘이 다하지!@% 않은 까닭입니다."], ["가슴 속에 하!@$나 둘 새겨지는 별을"]]

### 정규표현식을 사용하여 숫자, 특수문자 제거

In [23]:
preprocessed_texts = []

okt = Okt()

punctuation = ['.']

for sentence in sentences:
    
    s = sentence[0]

    #문장에서 특수문자나 숫자를 매칭함
    preprocessed_text = re.sub('[#$^&@!%0-9]','',s)

    #okt 토크나이저로 문장 토큰화, 구두점 제거
    tokenize_words = [word for word in okt.morphs(preprocessed_text) if word not in punctuation]

    preprocessed_texts.append(tokenize_words)

In [24]:
preprocessed_texts

[['안녕하세요', '음성', 'AI', '실습', '에', '오신', '것', '을', '환영', '합니다'],
 ['이', '네', '들', '은', '너무나', '멀리', '있습니다'],
 ['계절', '이', '지나가는', '하늘', '에는', '가을로', '가득', '차', '있습니다'],
 ['아직', '나', '의', '청춘', '이', '다', '하지', '않은', '까닭', '입니다'],
 ['가슴', '속', '에', '하나', '둘', '새겨지는', '별', '을']]

In [25]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(preprocessed_texts)

In [26]:
tokenizer.word_index

{'이': 1,
 '에': 2,
 '을': 3,
 '있습니다': 4,
 '안녕하세요': 5,
 '음성': 6,
 'ai': 7,
 '실습': 8,
 '오신': 9,
 '것': 10,
 '환영': 11,
 '합니다': 12,
 '네': 13,
 '들': 14,
 '은': 15,
 '너무나': 16,
 '멀리': 17,
 '계절': 18,
 '지나가는': 19,
 '하늘': 20,
 '에는': 21,
 '가을로': 22,
 '가득': 23,
 '차': 24,
 '아직': 25,
 '나': 26,
 '의': 27,
 '청춘': 28,
 '다': 29,
 '하지': 30,
 '않은': 31,
 '까닭': 32,
 '입니다': 33,
 '가슴': 34,
 '속': 35,
 '하나': 36,
 '둘': 37,
 '새겨지는': 38,
 '별': 39}

In [29]:
encoding_sentences = tokenizer.texts_to_sequences(preprocessed_texts)

In [30]:
encoding_sentences

[[5, 6, 7, 8, 2, 9, 10, 3, 11, 12],
 [1, 13, 14, 15, 16, 17, 4],
 [18, 1, 19, 20, 21, 22, 23, 24, 4],
 [25, 26, 27, 28, 1, 29, 30, 31, 32, 33],
 [34, 35, 2, 36, 37, 38, 39, 3]]

결과는 다음과 같이 나와야 합니다.  


[[5, 6, 7, 2, 8, 9, 3, 10, 11],  
 [1, 12, 13, 14, 15, 16, 4],  
 [17, 1, 18, 19, 20, 21, 22, 23, 4],  
 [24, 25, 26, 27, 1, 28, 29, 30, 31, 32],  
 [33, 34, 2, 35, 36, 37, 38, 3]]  
 

## Req. 1-2 Multi-head self-attention 구조 익히기

위에서 전처리한 데이터를 가져와 아래 과정을 실행하면서 시퀀스 입력이 multi-head self attention으로 어떻게 모델링 되는지 파악하시오.

In [31]:
pad_id = 0
vocab_size = 40

data = encoding_sentences

In [ ]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [ ]:
data, max_len = padding(data)

In [ ]:
data

### Hyperparameter 세팅 및 embedding

In [ ]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [ ]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [ ]:
print(batch_emb)
print(batch_emb.shape)

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [ ]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [ ]:
w_0 = nn.Linear(d_model, d_model)

In [ ]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [ ]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

In [ ]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [ ]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

In [ ]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [ ]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

In [ ]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

## Req. 1-3 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [ ]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [ ]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [ ]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)